In [1]:
from IPython.utils import io

# with io.capture_output() as captured:
#   !pip3 install scikit-learn;

import numpy as np

from util import train_test_df, evaluate as evaluator, interaction_matrix
from mfactorization import mf, mf_default_config


def pca(M):
  '''
  reduces dimension of nxm matrix M to nx64
  '''

  from sklearn.decomposition import PCA

  R = PCA(64).fit_transform(M)

  return R

In [2]:
# 1.
df_train, df_test = train_test_df()
R = interaction_matrix(df_train)

# 2.
print("factorizing interaction matrix:")
Q, P = mf(R, **mf_default_config, print_step=3)
Rm = Q@P.T

# 3.
E = np.load('bert_item_encodings.npy')

# 3.a
Pa = pca(E)
Ra = Q @ Pa.T

# 3.b
Pb = pca(np.hstack((E, P)))
Rb = Q @ Pb.T

# 4.
evaluate = lambda r: evaluator(r, df_test) # syntax suger

# 4. MF
Em = evaluate(Rm)

# 4. BERT a
Ea = evaluate(Ra)
# 4. BERT b
Eb = evaluate(Rb)

# 4. Extra evaluations
E0 = evaluate(R)

Rt = interaction_matrix(df_test)
Et = evaluate(Rt)

factorizing interaction matrix:


4it [00:00, 36.90it/s]

err: 2.29e+04, lr: 0.0037999 

230it [00:04, 46.18it/s]

err: 9.16e+03, lr: 0.0071223 , last batch: 8


...

err: 5.91e+03, lr: 0.0033997 , last batch: 2


678it [00:17, 50.69it/s]

err: 5.32e+03, lr: 0.0021398 , last batch: 1


700it [00:17, 39.95it/s]


best error: 5.30e+03


Mistake in the problem: the corresponding matrix for items is P (not Q)

In [3]:
print("We expect R_train to have almost zero score (since it has zero value for almost every item)")
print(E0)
print()

print("We expect R_test to have almost 1 score (since it has memorized test dataset)")
print(Et)

print("R estimation (with matrix factorization)")
print(Em)

print("Replace Q with BERT's encoding")
print(Ea)

print("Concat BERT encoding")
print(Eb)

We expect R_train to have almost zero score (since it has zero value for almost every item)
{'accuracy': '1.59', 'recall': '1.58', 'ndcg': '0.02', 'rank correlation': '0.90'}

We expect R_test to have almost 1 score (since it has memorized test dataset)
{'accuracy': '100', 'recall': '99', 'ndcg': '1.00', 'rank correlation': '0.99'}
R estimation (with matrix factorization)
{'accuracy': '1.72', 'recall': '1.71', 'ndcg': '0.02', 'rank correlation': '0.03'}
Replace Q with BERT's encoding
{'accuracy': '0.61', 'recall': '0.61', 'ndcg': '0.01', 'rank correlation': '0.04'}
Concat BERT encoding
{'accuracy': '0.80', 'recall': '0.79', 'ndcg': '0.01', 'rank correlation': '-0.02'}


In [10]:
import pandas as pd

columns = E0.keys()
df = pd.DataFrame(data=(E0, Et, Em, Ea, Eb), columns=columns)

df.insert(0, 'method', [
    'R (train data)',
    'R (test data)',
    'matrix factorization',
    'BERT replace Q',
    'BERT concat with Q'
])

df

,method,accuracy,recall,ndcg,rank correlation
0,R (train data),1.59,1.58,0.02,0.90
1,R (test data),100,99,1.00,0.99
2,matrix factorization,1.72,1.71,0.02,0.03
3,BERT replace Q,0.61,0.61,0.01,0.04
4,BERT concat with Q,0.80,0.79,0.01,-0.02
